**ФОРМИРОВАНИЕ ИСТОРИЧЕСКОЙ БАЗЫ ДАННЫХ ИЗ TINKOFF-API**

<h1>СОДЕРЖАНИЕ<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#0.-Загружаем-библиотеки" data-toc-modified-id="0.-Загружаем-библиотеки-1">0. Загружаем библиотеки</a></span></li><li><span><a href="#1.-Писоединяемся-к-базе-данных-MySQL-на-удалённом-сервере" data-toc-modified-id="1.-Писоединяемся-к-базе-данных-MySQL-на-удалённом-сервере-2">1. Писоединяемся к базе данных MySQL на удалённом сервере</a></span></li><li><span><a href="#2.-Формируем-базу-данных-инструментов-(только-акции)" data-toc-modified-id="2.-Формируем-базу-данных-инструментов-(только-акции)-3">2. Формируем базу данных инструментов (только акции)</a></span><ul class="toc-item"><li><span><a href="#2.1.-Формируем-таблицу-ИНСТРУМЕНТОВ-(акций)-Московской-биржи-(и-прочие)-https://tinkoff.github.io/investAPI/instruments/#share" data-toc-modified-id="2.1.-Формируем-таблицу-ИНСТРУМЕНТОВ-(акций)-Московской-биржи-(и-прочие)-https://tinkoff.github.io/investAPI/instruments/#share-3.1">2.1. Формируем таблицу ИНСТРУМЕНТОВ (акций) Московской биржи (и прочие) <a href="https://tinkoff.github.io/investAPI/instruments/#share" rel="nofollow" target="_blank">https://tinkoff.github.io/investAPI/instruments/#share</a></a></span></li><li><span><a href="#2.2.-Формируем-таблицу-типов-акций-id_share_type-https://tinkoff.github.io/investAPI/instruments/#sharetype" data-toc-modified-id="2.2.-Формируем-таблицу-типов-акций-id_share_type-https://tinkoff.github.io/investAPI/instruments/#sharetype-3.2">2.2. Формируем таблицу типов акций id_share_type <a href="https://tinkoff.github.io/investAPI/instruments/#sharetype" rel="nofollow" target="_blank">https://tinkoff.github.io/investAPI/instruments/#sharetype</a></a></span></li><li><span><a href="#2.3.-Формируем-таблицу-типов-статусов-акций-id_share_type-https://tinkoff.github.io/investAPI/instruments/#securitytradingstatus" data-toc-modified-id="2.3.-Формируем-таблицу-типов-статусов-акций-id_share_type-https://tinkoff.github.io/investAPI/instruments/#securitytradingstatus-3.3">2.3. Формируем таблицу типов статусов акций id_share_type <a href="https://tinkoff.github.io/investAPI/instruments/#securitytradingstatus" rel="nofollow" target="_blank">https://tinkoff.github.io/investAPI/instruments/#securitytradingstatus</a></a></span></li></ul></li><li><span><a href="#3.-Формируем-базу-данных-свечей" data-toc-modified-id="3.-Формируем-базу-данных-свечей-4">3. Формируем базу данных свечей</a></span><ul class="toc-item"><li><span><a href="#3.1.-Формируем-данные-по-историческим-5-ти-минутным-свечам" data-toc-modified-id="3.1.-Формируем-данные-по-историческим-5-ти-минутным-свечам-4.1">3.1. Формируем данные по историческим 5-ти минутным свечам</a></span></li><li><span><a href="#3.1.1.-Выгрузки-для-Гоши" data-toc-modified-id="3.1.1.-Выгрузки-для-Гоши-4.2">3.1.1. Выгрузки для Гоши</a></span></li><li><span><a href="#3.1.2.-Заполняем-базу--MySQL-на-удалённом-сервере" data-toc-modified-id="3.1.2.-Заполняем-базу--MySQL-на-удалённом-сервере-4.3">3.1.2. Заполняем базу  MySQL на удалённом сервере</a></span></li></ul></li><li><span><a href="#4.-16/09/23-Запрашиваем-торговый-календарь" data-toc-modified-id="4.-16/09/23-Запрашиваем-торговый-календарь-5">4. 16/09/23 Запрашиваем торговый календарь</a></span></li></ul></div>

# 0. Загружаем библиотеки

In [1]:
# Общие библиотеки
import os
from datetime import timedelta
import matplotlib.pyplot as plt
import time
import sqlite3
import pprint
from datetime import timedelta, datetime
import time

import pandas as pd
  ## Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
  ## Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
  ## Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

# Библиотека для работы с удалённой базой MySQL
# !python -m pip install mysql-connector 
# Источник: https://pythonpip.ru/examples/nastroyka-sredy-mysql-python-i-ustanovka-mysql-connector
import mysql.connector
# Параметры хоста
host = '91.106.207.34'
user = 'asgitma8_exchang'
password = 'DataBase2023'
db_name = 'asgitma8_exchang'

# Тинькофф-API
from tinkoff.invest import CandleInterval, Client, GetOrderBookRequest
from tinkoff.invest.utils import now

from tinkoff.invest.retrying.settings import RetryClientSettings
from tinkoff.invest.retrying.sync.client import RetryingClient

# Токин из спецфайла
from exchange.token import TOKEN

In [70]:
# Функция Выбора актуальных инструментов через TinkoffAPI
def get_instruments(EXCHANGE = 1): # Признак биржи: 0 - не определён, 1 - Московская, 2 - Притерская, 3 - вне биржи
    with Client(TOKEN) as client:
        response = client.instruments.shares().instruments
        rows = list()
        for elem in response:
            if elem.real_exchange == EXCHANGE:
                row = [elem.name, elem.ticker, elem.lot,
                       elem.min_price_increment.units+elem.min_price_increment.nano*1e-9,
                       elem.sector, elem.exchange,
                              
                       elem.nominal.currency, elem.nominal.units+elem.nominal.nano*1e-9, elem.issue_size,
                              
                       elem.figi, elem.class_code, elem.isin, elem.uid, elem.position_uid,
                              
                       int(elem.share_type), int(elem.trading_status),
                              
                       elem.short_enabled_flag, elem.buy_available_flag, elem.sell_available_flag,
                       elem.div_yield_flag, elem.api_trade_available_flag]

                rows.append(row)
            columns=['name', 'ticker', 'lot',
                                       'min_price_increment',
                                       'sector', 'exchange',
                                       
                                       'nominal_currency', 'nominal_value', 'issue_size',
                              
                                       'figi', 'class_code', 'isin', 'uid', 'position_uid',
                                       
                                       'id_share_type', 'id_trading_status',
                              
                                       'short_enabled_flag', 'buy_available_flag', 'sell_available_flag',
                                       'div_yield_flag', 'api_trade_available_flag']
            
            df = pd.DataFrame(columns=columns, data=rows)
    return df, rows, columns

In [112]:
# Функция загрузки в excel-файл исторических свечей по тикеру
def get_historical_candles_tikers(figis_tickers):
     with RetryingClient(TOKEN, settings= retry_settings) as client:
        for ticker, figi in zip(df_figi_gosha['ticker'], df_figis['figi']):
            new_string = dict()
            df_hisrory_candles = pd.DataFrame()
            for candle in client.get_all_candles(figi=figi,
                                                 from_=now() - timedelta(days=180),
                                                 interval=CandleInterval.CANDLE_INTERVAL_5_MIN):
                if candle.is_complete:
                    new_string = {'data': candle.time.date().strftime('%d-%m-%Y'),
                                  'time': candle.time.time().strftime('%H:%M:%S'),
                                  'open': round(candle.open.units+candle.open.nano*1e-9, 2),
                                  'high': round(candle.high.units+candle.high.nano*1e-9, 2),
                                  'low': round(candle.low.units+candle.low.nano*1e-9, 2),
                                  'close': round(candle.close.units+candle.close.nano*1e-9, 2),
                                  'volume': candle.volume}
                    df_hisrory_candles = df_hisrory_candles._append(new_string, ignore_index=True)
            
get_historical_candles_tikers(figis_tickers)

NameError: name 'retry_settings' is not defined

In [5]:
# Функция получения актуальных figi из базы данных
def get_figis_tickers(host, user, password, db_name):
    query = 'SELECT ticker, figi from shares'
    # Создаём подключение 
    myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)
    # Создаём курсора 
    cur = myconn.cursor() 
    try: 
        # Выполняем запрос к базе данных 
        cur.execute(query)
        # Выгружаем результаты запроса 
        figis_tickers = cur.fetchall() 
    except:
        # Откатываем изменения в случае ошибки соединения с базой
        myconn.rollback()
        print('Ошибка при работе с базой!')
    # Закрываем соединение с базой
    myconn.close()
    return figis_tickers

In [48]:
# Функция создания таблиц в базе данных под тикер
def creat_table_ticker(ticker, host, user, password, db_name):
    if ticker == 'VEON-RX': ticker = 'VEON_RX'
    query = f"CREATE TABLE `{ticker}` (                                   "\
            f"           `datetime` datetime NOT NULL,                    "\
            f"           `open`     decimal(12,6) DEFAULT NULL,           "\
            f"           `high`     decimal(12,6) DEFAULT NULL,           "\
            f"           `low`      decimal(12,6) DEFAULT NULL,           "\
            f"           `close`    decimal(12,6) DEFAULT NULL,           "\
            f"           `volume`   bigint(18) DEFAULT NULL,              "\
            f"           UNIQUE KEY `datetime_UNIQUE` (`datetime`)        "\
            f"                        ) ENGINE=InnoDB DEFAULT CHARSET=utf8"
    # Создаём подключение 
    myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)
    # Создаём курсора 
    cur = myconn.cursor() 
    try: 
        # Выполняем запрос к базе данных 
        cur.execute(query)
        # Фиксация данных в базе
        # myconn.commit() 
        print(f'Таблица {ticker} создана! ', end='') 
    except:
        # Откатываем изменения в случае ошибки соединения с базой
        myconn.rollback()
        print('Ошибка при работе с базой!')
    # Закрываем соединение с базой
    myconn.close()

In [41]:
# Функция загрузки свечей по параметру figi
def get_historical_candles_figi(figi):
    retry_settings = RetryClientSettings(use_retry=True, max_retry_attempt=2)
    with RetryingClient(TOKEN, settings=retry_settings) as client:
        rows = list()
        for candle in client.get_all_candles(figi=figi, from_=now() - timedelta(days=182),
                                             interval=CandleInterval.CANDLE_INTERVAL_5_MIN):
            row = tuple([candle.time,#.strftime('%d-%m-%Y %H:%M:%S.%f'),
                        candle.open.units+candle.open.nano*1e-9,
                        candle.high.units+candle.high.nano*1e-9,
                        candle.low.units+candle.low.nano*1e-9,
                        candle.close.units+candle.close.nano*1e-9,
                        candle.volume])
            rows.append(row)
        print(f'Получено {len(rows)} свечей!', end=' ')
    return rows

In [50]:
# Функция заполнения таблицы для тикера
def insert_table_ticker(ticker, rows):
    if ticker == 'VEON-RX': ticker = 'VEON_RX'
    # Формируем запрос-вставку в базу данных
    
    sql =   f'insert into {ticker}(datetime, open, high, low, close, volume) '\
            f'values(%s, %s, %s, %s, %s, %s)'
    # Создаём подключение 
    myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)
    # Создаём курсора 
    cur = myconn.cursor()
    try: 
        # Выполняем вставку в базу данных 
        cur.executemany(sql, rows)
        # Фиксация данных в базе
        myconn.commit() 
        print(f'{ticker} внесено {cur.rowcount} записей!')
    except:
        # Откатываем изменения в случае ошибки соединения с базой
        myconn.rollback()
        print('Ошибка при работе с базой!')

    # Закрываем соединение с базой
    myconn.close()

In [22]:
# Функция проверки отсутствия некорректных данных в таблице
def examination_datatables(ticker):
     # Формируем запрос-вставку в базу данных
    sql =   f"select count(*) from {ticker} where open='9999.999999'"
    # Создаём подключение 
    myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)
    # Создаём курсора 
    cur = myconn.cursor() 
    try: 
        # Выполняем запрос к базе данных 
        cur.execute(sql)
        # Выгружаем результаты запроса 
        result = cur.fetchall() 
        if result:
            print(f'{ticker} записей {result}')
    except:
        # Откатываем изменения в случае ошибки соединения с базой
        myconn.rollback()
        print('Ошибка при работе с базой!')

    # Закрываем соединение с базой
    myconn.close()

In [52]:
# Создадим в базе данных таблицы под каждый тикер и загрузим в них исторические свечи за 180 дней

# # Получаем список figi и tickere
figis_tickers = [('VEON-RX', 'BBG00R4Z2NT4')]

# Перебираем пары:
count = 1
for ticker, figi in figis_tickers:
    print(f'{count}.', end=' ')
    # Создаём таблицу тикера
    creat_table_ticker(ticker, host, user, password, db_name)
    # Выбираем свечи для тикера за 180 дней
    rows = get_historical_candles_figi(figi=figi)
    # Заполняем таблицу базы данных свечами
    insert_table_ticker(ticker, rows)
    count += 1

1. Таблица VEON_RX создана! Получено 8612 свечей! VEON_RX внесено 8612 записей!


In [43]:
# Проверка заполненности таблиц
figis_tickers = get_figis_tickers(host, user, password, db_name)
for ticker, figi in figis_tickers:
    examination_datatables(ticker)

DSKY записей [(0,)]
ENPG записей [(0,)]
POLY записей [(0,)]
NKNCP записей [(0,)]
IRKT записей [(0,)]
MRKV записей [(0,)]
VSMO записей [(0,)]
UNAC записей [(0,)]
LSRG записей [(0,)]
TTLK записей [(0,)]
CNTL записей [(0,)]
MGNT записей [(0,)]
SVAV записей [(0,)]
ETLN записей [(0,)]
RTKMP записей [(0,)]
GEMC записей [(0,)]
KZOSP записей [(0,)]
WUSH записей [(0,)]
KMAZ записей [(0,)]
KZOS записей [(0,)]
FIXP записей [(0,)]
VKCO записей [(0,)]
PHOR записей [(0,)]
HHRU записей [(0,)]
LNZL записей [(0,)]
MGTSP записей [(0,)]
ELFV записей [(0,)]
SELG записей [(0,)]
AGRO записей [(0,)]
TATNP записей [(0,)]
Ошибка при работе с базой!
BLNG записей [(0,)]
PRFN записей [(0,)]
MAGN записей [(0,)]
VTBR записей [(0,)]
SMLT записей [(0,)]
BANE записей [(0,)]
AFLT записей [(0,)]
CIAN записей [(0,)]
CARM записей [(0,)]
RUAL записей [(0,)]
NKHP записей [(0,)]
SBER записей [(0,)]
GECO записей [(0,)]
MDMG записей [(0,)]
GTRK записей [(0,)]
MVID записей [(0,)]
BANEP записей [(0,)]
OKEY записей [(0,)]
ALRS за

In [44]:
figis_tickers

[('DSKY', 'BBG000BN56Q9'),
 ('ENPG', 'BBG000RMWQD4'),
 ('POLY', 'BBG004PYF2N3'),
 ('NKNCP', 'BBG000GQSVC2'),
 ('IRKT', 'BBG000FWGSZ5'),
 ('MRKV', 'BBG000PKWCQ7'),
 ('VSMO', 'BBG004S68CV8'),
 ('UNAC', 'BBG000Q7ZZY2'),
 ('LSRG', 'BBG004S68C39'),
 ('TTLK', 'BBG000RJL816'),
 ('CNTL', 'BBG000BBV4M5'),
 ('MGNT', 'BBG004RVFCY3'),
 ('SVAV', 'BBG004S68JR8'),
 ('ETLN', 'BBG001M2SC01'),
 ('RTKMP', 'BBG004S685M3'),
 ('GEMC', 'BBG011MCM288'),
 ('KZOSP', 'BBG0029SG1C1'),
 ('WUSH', 'TCS00A105EX7'),
 ('KMAZ', 'BBG000LNHHJ9'),
 ('KZOS', 'BBG0029SFXB3'),
 ('FIXP', 'BBG00ZHCX1X2'),
 ('VKCO', 'BBG00178PGX3'),
 ('PHOR', 'BBG004S689R0'),
 ('HHRU', 'TCS2207L1061'),
 ('LNZL', 'BBG000SK7JS5'),
 ('MGTSP', 'BBG000PZ0833'),
 ('ELFV', 'BBG000F6YPH8'),
 ('SELG', 'BBG002458LF8'),
 ('AGRO', 'BBG007N0Z367'),
 ('TATNP', 'BBG004S68829'),
 ('VEON-RX', 'BBG00R4Z2NT4'),
 ('BLNG', 'BBG000VQWH86'),
 ('PRFN', 'TCS00A0JNXF9'),
 ('MAGN', 'BBG004S68507'),
 ('VTBR', 'BBG004730ZJ9'),
 ('SMLT', 'BBG00F6NKQX3'),
 ('BANE', 'BBG004S68

# 1. Писоединяемся к базе данных MySQL на удалённом сервере

In [41]:
# Формируем запросы
query = 'SELECT VERSION()'
query1 = "SHOW VARIABLES LIKE '%version%'"
query2 = 'SHOW DATABASES'
query3 = 'SHOW TABLES'
query4 = 'SELECT * FROM share_type'
query5 = 'SELECT * FROM trading_status'
query6 = 'SELECT * FROM shares'

In [43]:
# Создаём подключение 
myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)
# Создаём курсора 
cur = myconn.cursor() 
try: 
    # Выполняем запрос к базе данных 
    cur.execute(query3)
    # Выгружаем результаты запроса 
    result = cur.fetchall() 
    # Печатаем результаты запроса
    for x in result: 
        print(x); 
except:
    # Откатываем изменения в случае ошибки соединения с базой
    myconn.rollback() 
# Закрываем соединение с базой
myconn.close()

('share_type',)
('shares',)
('trading_status',)


In [26]:
tickers = [('VSMO',), ('LNZL',), ('AKRN',),
           ('TRNFP',), ('RKKE',), ('CHMK',),
           ('UNKL',), ('GMKN',), ('PLZL',),
           ('KRKNP',)]

In [31]:
# Создаём подключение 
myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)
# Создаём курсора 
cur = myconn.cursor()
# Создаём запрос
sql = f"select ticker, figi from shares "\
      f"where ticker in ('VSMO', 'LNZL', 'AKRN', 'TRNFP', 'RKKE' , 'CHMK', 'UNKL', 'GMKN', 'PLZL', 'KRKNP')"
try: 
    # Выполняем запрос к базе данных 
    cur.execute(sql)
    # Выгружаем результаты запроса 
    result = cur.fetchall() 
    # Печатаем результаты запроса
    print(result); 
except:
    # Откатываем изменения в случае ошибки соединения с базой
    myconn.rollback()
    print('Ошибка подключения к базе данных')
# Закрываем соединение с базой
myconn.close()

[('AKRN', 'BBG004S688G4'), ('CHMK', 'BBG000RP8V70'), ('GMKN', 'BBG004731489'), ('KRKNP', 'BBG002B2J5X0'), ('LNZL', 'BBG000SK7JS5'), ('PLZL', 'BBG000R607Y3'), ('RKKE', 'BBG000LWNRP3'), ('TRNFP', 'BBG00475KHX6'), ('UNKL', 'BBG002YFXL29'), ('VSMO', 'BBG004S68CV8')]


In [37]:
 tickers = [('VSMO',), ('LNZL',), ('AKRN',),
                 ('TRNFP',), ('RKKE',), ('CHMK',),
                 ('UNKL',), ('GMKN',), ('PLZL',),
                 ('KRKNP',)] #get_figis_tickers(host, user, password, db_name)
tickers.sort()

In [38]:
tickers

[('AKRN',),
 ('CHMK',),
 ('GMKN',),
 ('KRKNP',),
 ('LNZL',),
 ('PLZL',),
 ('RKKE',),
 ('TRNFP',),
 ('UNKL',),
 ('VSMO',)]

# 2. Формируем базу данных инструментов (только акции)

![](./Pictures/trading_instrument_dentifiers.png)

![](./Pictures/financial_instrument_identifiers.png)

## 2.1. Формируем таблицу ИНСТРУМЕНТОВ (акций) Московской биржи (и прочие) https://tinkoff.github.io/investAPI/instruments/#share

In [28]:
# Расположение базы данных и отчётов
data_path = './Data/'

In [71]:
# Посчитаем сколько и какие акции обращаются на какой бирже
# Признак биржи: 0 - не определён, 1 - Московская, 2 - Притерская, 3 - вне биржи
for EXCHANGE in [0, 1, 2, 3]:
    df, rows, columns = get_instruments(EXCHANGE)
    if EXCHANGE == 2:
        df.to_excel(f'{data_path}Питерская биржа на Тинькофф ({len(df)} акций).xlsx', index=False)
    elif EXCHANGE == 1:
        COUNT_SHARE_MOEX = len(df)
        df_MOEX = df.copy()
        rows_MOEX = rows
        columns_MOEX = columns
        df_MOEX.to_excel(f'{data_path}Московская биржа на Тинькофф ({COUNT_SHARE_MOEX} акций).xlsx', index=False)
    elif EXCHANGE == 3:
        df.to_excel(f'{data_path}Вне биржи на Тинькофф ({len(df)} акций).xlsx', index=False)
    else:
        df.to_excel(f'{data_path}Биржа не определена на Тинькофф ({len(df)} акций).xlsx', index=False)
del df, rows, columns

In [72]:
df_MOEX.head(5)

,name,ticker,lot,min_price_increment,sector,exchange,nominal_currency,nominal_value,issue_size,figi,class_code,isin,uid,position_uid,id_share_type,id_trading_status,short_enabled_flag,buy_available_flag,sell_available_flag,div_yield_flag,api_trade_available_flag
0,Детский Мир,DSKY,10,0.02,consumer,MOEX,rub,0.00040,739000000,BBG000BN56Q9,TQBR,RU000A0JSQ90,6e061639-6198-4448-9568-1eadb1b0e127,e07311a4-fac4-472b-b1d7-c3a7b3e56a6d,1,1,False,True,True,True,True
1,En+ Group,ENPG,1,0.20,materials,MOEX_EVENING_WEEKEND,usd,0.00007,638848896,BBG000RMWQD4,TQBR,RU000A100K72,e2bd2eba-75de-4127-b39c-2f2dbe3866c3,62dae90c-238b-433d-ae4e-47c72b324bc7,1,1,False,True,True,False,True
2,Polymetal,POLY,1,0.10,materials,MOEX_PLUS,usd,0.03000,473626239,BBG004PYF2N3,TQBR,JE00B6T5S470,127361c2-32ec-448c-b3ec-602166f537ea,f869a0b6-e4cd-4e15-a25a-095c936f2e3f,1,1,False,True,True,True,True
3,Нижнекамскнефтехим - акции привилегированные,NKNCP,10,0.02,materials,MOEX,rub,1.00000,218983750,BBG000GQSVC2,TQBR,RU0006765096,bc21fb4f-8838-4355-8697-fb0d8fc809c8,7b7201cb-fe73-49a8-95d6-bd5edf4b849c,2,1,True,True,True,True,True
4,Яковлев,IRKT,100,0.05,industrials,MOEX,rub,3.00000,2085163865,BBG000FWGSZ5,TQBR,RU0006752979,cfb50a23-2465-497e-bc7e-e4f0e042cf3d,1a583ddf-b4af-426b-890f-2749767a8379,1,1,False,True,True,True,True


In [39]:
# Оптимизируем используемую память отчищая массив с акциями на Московской бирже
del df_MOEX

In [32]:
# Акции, не вошедшие в API_Tinkoff, которые есть в списке Стёпы
df_stepa = pd.read_excel(f'{data_path}Тикеры ММВБ.xlsx')
non_in_list_Tinkoff = set(df_stepa['Тикер']) - set(pd.
                     read_excel(f'{data_path}Московская биржа на Тинькофф ({COUNT_SHARE_MOEX} акций).xlsx')['ticker'])
df_delta_stepa = df_stepa[df_stepa['Тикер'].isin(non_in_list_Tinkoff)]
df_delta_stepa.to_excel(f'{data_path}Есть у Стёпы и нет у Тинкофф ({len(df_delta_stepa)} акций).xlsx', index=False)

In [33]:
# Акции, не вошедшие в API_Tinkoff, которые есть в списке Гоши
df_gosha = pd.read_excel(f'{data_path}Список инструментов для анализа.xlsx', header=1)
non_in_list_Tinkoff_gosha = set(df_gosha['Тикер']) - set(pd.
                     read_excel(f'{data_path}Московская биржа на Тинькофф ({COUNT_SHARE_MOEX} акций).xlsx')['ticker'])
df_delta_gosha = df_gosha[df_gosha['Тикер'].isin(non_in_list_Tinkoff_gosha)][['Тикер', 'Ссылка на TradingViw']]
df_delta_gosha.to_excel(f'{data_path}Есть у Гоши и нет у Тинкофф ({len(df_delta_gosha)} акций).xlsx', index=False)

In [82]:
# Вносим данные таблицы инструментов (акций) Мосбиржы в удалённую базу данных
# Формируем запрос-вставку в базу данных
col = ', '.join(columns_MOEX)
val = [tuple(elem) for elem in rows_MOEX]
sql =   f'insert into shares({col}) '\
        f'values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [81]:
# !!!!! Уже заполнено ДАННЫЙ КОД НЕЛЬЗЯ ПОВТОРЯТЬ БЕЗ УДАЛЕНИЯ ПРЕДЫДУЩИХ СТРОК В УДАЛЁННОЙ БАЗЕ ДАННЫХ
# # Создаём подключение 
# myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)

# # Создаём курсора 
# cur = myconn.cursor()
     
# try: 
#     # Выполняем вставку в базу данных 
#     cur.executemany(sql, val)
 
#     # Фиксация данных в базе
#     myconn.commit() 
#     print(cur.rowcount," Все записи внесены!")
    
# except:
#     # Откатываем изменения в случае ошибки соединения с базой
#     myconn.rollback()

# # Закрываем соединение с базой
# myconn.close()

140  Все записи внесены!


## 2.2. Формируем таблицу типов акций id_share_type https://tinkoff.github.io/investAPI/instruments/#sharetype

In [182]:
df_share_type = pd.DataFrame({'id_share_type': [0, 1, 2, 3, 4, 5, 6, 7, 8],
                              'name': ['SHARE_TYPE_UNSPECIFIED', 'SHARE_TYPE_COMMON', 'SHARE_TYPE_PREFERRED',
                                       'SHARE_TYPE_ADR', 'SHARE_TYPE_GDR', 'SHARE_TYPE_MLP',
                                       'SHARE_TYPE_NY_REG_SHRS', 'SHARE_TYPE_CLOSED_END_FUND',
                                       'SHARE_TYPE_REIT'],
                              'description': ['Значение не определено', 'Обыкновенная',
                                              'Привилегированная',
                                              'Американские депозитарные расписки',
                                              'Глобальные депозитарные расписки',
                                              'Товарищество с ограниченной ответственностью',
                                              'Акции из реестра Нью-Йорка', 'Закрытый инвестиционный фонд',
                                              'Траст недвижимости']
                             })

In [183]:
### !!! Даная таблица перенесена в базу данных без автоматических процедур
df_share_type

,id_share_type,name,description
0,0,SHARE_TYPE_UNSPECIFIED,Значение не определено
1,1,SHARE_TYPE_COMMON,Обыкновенная
2,2,SHARE_TYPE_PREFERRED,Привилегированная
3,3,SHARE_TYPE_ADR,Американские депозитарные расписки
4,4,SHARE_TYPE_GDR,Глобальные депозитарные расписки
5,5,SHARE_TYPE_MLP,Товарищество с ограниченной ответственностью
6,6,SHARE_TYPE_NY_REG_SHRS,Акции из реестра Нью-Йорка
7,7,SHARE_TYPE_CLOSED_END_FUND,Закрытый инвестиционный фонд
8,8,SHARE_TYPE_REIT,Траст недвижимости


## 2.3. Формируем таблицу типов статусов акций id_share_type https://tinkoff.github.io/investAPI/instruments/#securitytradingstatus

In [184]:
df_trading_status = pd.DataFrame({'id_trading_status': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                    11, 12, 13, 14, 15, 16],
                                  'name': ['SECURITY_TRADING_STATUS_UNSPECIFIED',
                                           'SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING',
                                           'SECURITY_TRADING_STATUS_OPENING_PERIOD',
                                           'SECURITY_TRADING_STATUS_CLOSING_PERIOD',
                                           'SECURITY_TRADING_STATUS_BREAK_IN_TRADING',
                                           'SECURITY_TRADING_STATUS_NORMAL_TRADING',
                                           'SECURITY_TRADING_STATUS_CLOSING_AUCTION',
                                           'SECURITY_TRADING_STATUS_DARK_POOL_AUCTION',
                                           'SECURITY_TRADING_STATUS_DISCRETE_AUCTION',
                                           'SECURITY_TRADING_STATUS_OPENING_AUCTION_PERIOD',
                                           'SECURITY_TRADING_STATUS_TRADING_AT_CLOSING_AUCTION_PRICE',
                                           'SECURITY_TRADING_STATUS_SESSION_ASSIGNED',
                                           'SECURITY_TRADING_STATUS_SESSION_CLOSE',
                                           'SECURITY_TRADING_STATUS_SESSION_OPEN',
                                           'SECURITY_TRADING_STATUS_DEALER_NORMAL_TRADING',
                                           'SECURITY_TRADING_STATUS_DEALER_BREAK_IN_TRADING',
                                           'SECURITY_TRADING_STATUS_DEALER_NOT_AVAILABLE_FOR_TRADING'],
                                  'description': ['Торговый статус не определён', 'Недоступен для торгов',
                                                  'Период открытия торгов', 'Период закрытия торгов',
                                                  'Перерыв в торговле', 'Нормальная торговля',
                                                  'Аукцион закрытия', 'Аукцион крупных пакетов',
                                                  'Дискретный аукцион', 'Аукцион открытия',
                                                  'Период торгов по цене аукциона закрытия',
                                                  'Сессия назначена', 'Сессия закрыта', 'Сессия открыта',
                                                  'Доступна торговля в режиме внутренней ликвидности брокера',
                                                  'Перерыв торговли в режиме внутренней ликвидности брокера',
                                                  'Недоступна торговля в режиме внутренней ликвидности брокера']
                                 })

In [185]:
### !!! Даная таблица перенесена в базу данных без автоматических процедур
df_trading_status

,id_trading_status,name,description
0,0,SECURITY_TRADING_STATUS_UNSPECIFIED,Торговый статус не определён
1,1,SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING,Недоступен для торгов
2,2,SECURITY_TRADING_STATUS_OPENING_PERIOD,Период открытия торгов
3,3,SECURITY_TRADING_STATUS_CLOSING_PERIOD,Период закрытия торгов
4,4,SECURITY_TRADING_STATUS_BREAK_IN_TRADING,Перерыв в торговле
5,5,SECURITY_TRADING_STATUS_NORMAL_TRADING,Нормальная торговля
6,6,SECURITY_TRADING_STATUS_CLOSING_AUCTION,Аукцион закрытия
7,7,SECURITY_TRADING_STATUS_DARK_POOL_AUCTION,Аукцион крупных пакетов
8,8,SECURITY_TRADING_STATUS_DISCRETE_AUCTION,Дискретный аукцион
9,9,SECURITY_TRADING_STATUS_OPENING_AUCTION_PERIOD,Аукцион открытия


In [84]:
# Оптимизируем память
dfs = []
local_variable_scope =locals().keys()
for elem in local_variable_scope:
    if 'df' in elem:
        dfs.append(elem)
# Удаляем не используемые датафреймы
for elem in dfs:
    del locals()[elem]

# 3. Формируем базу данных свечей

## 3.1. Формируем данные по историческим 5-ти минутным свечам

## 3.1.1. Выгрузки для Гоши

In [234]:
# Формирование исторических свечей для Гоши

# Перечень тикеров, которые есть в Тиньков и есть в списке Гоши
ticker_list_Tinkoff_gosha = list(set(df_gosha['Тикер']) - set(non_in_list_Tinkoff_gosha))
df_tinkoff = pd.read_excel(f'{data_path}Московская биржа на Тинькофф ({COUNT_SHARE_MOEX} акций).xlsx')
df_figi_gosha = df_tinkoff[df_tinkoff.ticker.isin(ticker_list_Tinkoff_gosha)][['ticker', 'figi']].to_dict('list')

## 3.1.2. Заполняем базу  MySQL на удалённом сервере

# 4. 16/09/23 Запрашиваем торговый календарь

In [248]:
# Запросим актуальный перечень акций, торгуемых в Тинькофф на Московской бирже
with Client(TOKEN) as client:
    shares = client.instruments.shares().instruments

In [251]:
# Определим перечень типов расписаний торгуемых акций
exchanges = set()
for elem in shares:
    if int(elem.real_exchange) == 1:
        exchanges.add(elem.exchange)

In [252]:
# Сформируем расписания торгов для акций с разными расписаниями
for exchange in list(exchanges):
    with Client(TOKEN) as client:
        response = client.instruments.trading_schedules(exchange=exchange,
                                                        from_=now()+timedelta(days=2),
                                                        to=now()+timedelta(days=2))
    print(f'{exchange}:\n'
          f'  - время начала торгов {response.exchanges[0].days[0].start_time}\n'
          f'  - время окончания торгов {response.exchanges[0].days[0].end_time}')

MOEX_PLUS:
  - время начала торгов 2023-09-18 07:00:00+00:00
  - время окончания торгов 2023-09-18 20:50:00+00:00
MOEX_EVENING_WEEKEND:
  - время начала торгов 2023-09-18 07:00:00+00:00
  - время окончания торгов 2023-09-18 20:49:59+00:00
MOEX_WEEKEND:
  - время начала торгов 2023-09-18 07:00:00+00:00
  - время окончания торгов 2023-09-18 15:39:59+00:00
MOEX:
  - время начала торгов 2023-09-18 07:00:00+00:00
  - время окончания торгов 2023-09-18 15:39:59+00:00


In [260]:
with Client(TOKEN) as client:
    response = client.instruments.trading_schedules(exchange=exchange,
                                                    from_=now()+timedelta(days=2),
                                                    to=now()+timedelta(days=2))

In [266]:
response.exchanges[0].days[0].__dict__

{'date': datetime.datetime(2023, 9, 18, 0, 0, tzinfo=datetime.timezone.utc),
 'is_trading_day': True,
 'start_time': datetime.datetime(2023, 9, 18, 7, 0, tzinfo=datetime.timezone.utc),
 'end_time': datetime.datetime(2023, 9, 18, 15, 39, 59, tzinfo=datetime.timezone.utc),
 'opening_auction_start_time': datetime.datetime(2023, 9, 18, 6, 50, tzinfo=datetime.timezone.utc),
 'closing_auction_end_time': datetime.datetime(2023, 9, 18, 15, 50, tzinfo=datetime.timezone.utc),
 'evening_opening_auction_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'evening_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'evening_end_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'clearing_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'clearing_end_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'premarket_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzin

In [210]:
response.exchanges[0].days[0].__dict__

{'date': datetime.datetime(2023, 9, 18, 0, 0, tzinfo=datetime.timezone.utc),
 'is_trading_day': True,
 'start_time': datetime.datetime(2023, 9, 18, 7, 0, tzinfo=datetime.timezone.utc),
 'end_time': datetime.datetime(2023, 9, 18, 15, 39, 59, tzinfo=datetime.timezone.utc),
 'opening_auction_start_time': datetime.datetime(2023, 9, 18, 6, 50, tzinfo=datetime.timezone.utc),
 'closing_auction_end_time': datetime.datetime(2023, 9, 18, 15, 50, tzinfo=datetime.timezone.utc),
 'evening_opening_auction_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'evening_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'evening_end_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'clearing_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'clearing_end_time': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
 'premarket_start_time': datetime.datetime(1970, 1, 1, 0, 0, tzin

In [130]:
response.exchanges[0].days[0].is_trading_day,\
response.exchanges[0].days[0].start_time,\
response.exchanges[0].days[0].end_time, 

(True,
 datetime.datetime(2023, 9, 18, 7, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(2023, 9, 18, 15, 39, 59, tzinfo=datetime.timezone.utc))

In [133]:
if response.exchanges[0].days[0].is_trading_day:
    print(response.exchanges[0].days[0].start_time)
    print(response.exchanges[0].days[0].end_time)

2023-09-18 07:00:00+00:00
2023-09-18 15:39:59+00:00


In [195]:
with Client(TOKEN) as client:
        response = client.instruments.shares().instruments # [elem for elem in client.instruments.shares().instruments if elem.class_code=='MOEX']

In [198]:
exchanges = set()
for elem in response:
    if int(elem.real_exchange) == 1:
        exchanges.add(elem.exchange)

In [200]:
exchanges, type(exchanges)

({'MOEX', 'MOEX_EVENING_WEEKEND', 'MOEX_PLUS', 'MOEX_WEEKEND'}, set)

In [192]:
set([elem.exchange for elem in shares])

{'MOEX', 'MOEX_EVENING_WEEKEND', 'MOEX_PLUS', 'MOEX_WEEKEND'}

In [180]:
shares[0].__dict__

{'figi': 'BBG000BN56Q9',
 'ticker': 'DSKY',
 'class_code': 'TQBR',
 'isin': 'RU000A0JSQ90',
 'lot': 10,
 'currency': 'rub',
 'klong': Quotation(units=2, nano=0),
 'kshort': Quotation(units=2, nano=0),
 'dlong': Quotation(units=1, nano=0),
 'dshort': Quotation(units=1, nano=0),
 'dlong_min': Quotation(units=1, nano=0),
 'dshort_min': Quotation(units=1, nano=0),
 'short_enabled_flag': False,
 'name': 'Детский Мир',
 'exchange': 'MOEX',
 'ipo_date': datetime.datetime(2014, 2, 11, 0, 0, tzinfo=datetime.timezone.utc),
 'issue_size': 739000000,
 'country_of_risk': 'RU',
 'country_of_risk_name': 'Российская Федерация',
 'sector': 'consumer',
 'issue_size_plan': 739000000,
 'nominal': MoneyValue(currency='rub', units=0, nano=400000),
 'trading_status': <SecurityTradingStatus.SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING: 1>,
 'otc_flag': False,
 'buy_available_flag': True,
 'sell_available_flag': True,
 'div_yield_flag': True,
 'share_type': <ShareType.SHARE_TYPE_COMMON: 1>,
 'min_price_in

In [19]:
 # Формируем запрос-вставку в базу данных

sql = f"select count(*) from LNZL"#where open='9999.999999'"
# Создаём подключение 
myconn = mysql.connector.connect(host=host, user=user, passwd=password, database=db_name)
# Создаём курсора 
cur = myconn.cursor() 
# Выполняем запрос к базе данных 
cur.execute(sql)
# Выгружаем результаты запроса 
result = cur.fetchall() 
if result:
    print(f'{ticker} записей {result}')

KZOSP записей [(10749,)]


In [108]:
from tinkoff.invest import (CandleInstrument,
                            Client,
                            InfoInstrument,
                            SubscriptionInterval,)
from tinkoff.invest.services import MarketDataStreamManager

In [124]:
# Сформируем перечень
def get_strem_candles():
    with Client(TOKEN) as client:
        market_data_stream: MarketDataStreamManager = client.create_market_data_stream()
        market_data_stream.candles.unsubscribe(
            [
                CandleInstrument(
                    figi="BBG004730N88",
                    interval=SubscriptionInterval.SUBSCRIPTION_INTERVAL_FIVE_MINUTES,
                )
            ]
        )
        for marketdata in market_data_stream:
            print(marketdata)
            market_data_stream.info.subscribe([InfoInstrument(figi="BBG004730N88")])
            if marketdata..subscribe_info_response:
                market_data_stream.stop()

In [125]:
get_strem_candles()

/Users/work/opt/anaconda3/lib/python3.9/site-packages/google/protobuf/symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


ValueError: 9 is not a valid SubscriptionStatus

In [150]:
dfs = []
local_variable_scope =locals().keys()
for elem in local_variable_scope:
    if 'df' in elem:
        dfs.append(elem)
# Удаляем не используемые датафреймы
for elem in dfs:
    del locals()[elem]

In [7]:
data_path = './Data/'
# Заполняем датафрейм результатами запроса
connection = sqlite3.connect(f'{data_path}exchange')
print('База данных подключена')
# Загрузим датасет с заказами и временем доставки
df = pd.read_sql_query(query_0, connection)
# df = pd.read_sql_query(query_1, connection)
print('Датафрейм заполнен фичами и параметрами для расчёта целевой переменной')
connection.close()
print('Соединение с базой данных закрыто')

База данных подключена
Датафрейм заполнен фичами и параметрами для расчёта целевой переменной
Соединение с базой данных закрыто


In [8]:
df

,Признак закрытия,Время свечи,Объём в лотах,Цена открытия,Максимальная цена,Минимальная цена,Цена закрытия
0,1,1694097300,157544,256,257,256,256
1,1,1694097600,86803,256,257,256,257
2,1,1694097900,59178,257,257,256,257
3,1,1694098200,59391,257,257,257,257
4,1,1694098500,65565,257,257,256,256
...,...,...,...,...,...,...,...
159,1,1694182500,9550,256,256,256,256
160,1,1694182800,17943,256,256,256,256
161,1,1694183100,8455,256,256,256,256
162,1,1694183400,7765,256,256,256,256
